# Cythonized Entropy Function for Binwalk

## About

This notebook contains a proposed change to [Binwalk](https://github.com/devttys0/binwalk) that improves the performance of the Shannon entropy function.

In [2]:
%load_ext Cython

In [77]:
import math
s = ''.join(['test' for _ in range(10000000)])

In [67]:
# original implementation
def py_shannon(data):
    '''
    Performs a Shannon entropy analysis on a given block of data.
    '''
    entropy = 0

    if data:
        length = len(data)

        seen = dict(((chr(x), 0) for x in range(0, 256)))
        for byte in data:
            seen[byte] += 1

        for x in range(0, 256):
            p_x = float(seen[chr(x)]) / length
            if p_x > 0:
                entropy -= p_x * math.log(p_x, 2)

    return (entropy / 8)

In [1]:
%%cython 
import cython
from libc.string cimport memset
from libc.math cimport log2

# cythonized
def shannon(data):
    '''
    Performs a Shannon entropy analysis on a given block of data.
    '''
    cdef:
        double entropy = 0
        int seen[256]
        float length
        double p_x
        char* c
        int x
            
    memset(seen, 0, 256 * sizeof(int))
    
    if data:
        length = len(data)

        for c in data:
            seen[c[0]] += 1
            
        for x in range(0, 256):
            p_x = seen[x] / length
            if p_x > 0:
                entropy -= p_x * log2(p_x)

    return (entropy / 8)

## Benchmark

In [78]:
%timeit print(shannon(s))
%timeit print(py_shannon(s))

0.1875
0.1875
0.1875
0.1875
1 loop, best of 3: 682 ms per loop
0.1875
0.1875
0.1875
0.1875
1 loop, best of 3: 4.21 s per loop
